### импортируем все нужное

In [1]:
import re
import pandas as pd
import csv
from tqdm import tqdm
from redditscore.tokenizer import CrazyTokenizer

In [2]:
from nltk.corpus import stopwords
stops = stopwords.words('russian')

## Расширяем список стопслов

In [3]:
stops.extend(stopwords.words('english'))

In [4]:
badwords = [
'я', 'а', 'да', 'но', 'тебе', 'мне', 'ты', 'и', 'у', 'на', 'ща', 'ага',
'так', 'там', 'какие', 'который', 'какая', 'туда', 'давай', 'короче', 'кажется', 'вообще',
'ну', 'не', 'чет', 'неа', 'свои', 'наше', 'хотя', 'такое', 'например', 'кароч', 'как-то',
'нам', 'хм', 'всем', 'нет', 'да', 'оно', 'своем', 'про', 'вы', 'тд', 'тп', 'т.д', 'т.п',
'вся', 'вам', 'это', 'эта', 'эти', 'этот', 'прям', 'либо', 'как', 'мы',
'просто', 'блин', 'очень', 'самые', 'твоем', 'ваша', 'кстати', 'вроде', 'типа', 'пока', 'ок',
    '°', '͜ʖ', '͡', 'ツ', 'ʕ•ᴥ•ʔ'
]

In [5]:
stops.extend(badwords)

In [6]:
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

### Открываем данные

In [7]:
with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [8]:
df.head(5)

,tweet_id,tweet,class
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0


### выбираем тестовые твиты

In [9]:
test_data = [df['tweet'][8],
             df['tweet'][9], # дефисы
             df['tweet'][19],
             df['tweet'][268], # дефисы, капс
             
             df['tweet'][273], # латиница, нет пробелов вокруг скобки
             
             df['tweet'][10], # 0,5г ; р/д
             df['tweet'][20], # 125 мгк 2 раза ; Вентолин 100 мгк
            
             df['tweet'][2368], # нет пробела между предложениями
             
             df['tweet'][14], # кавычки, юзернейм, эмоджи
             df['tweet'][18], # хештеги, юзернейм
             df['tweet'][25], # смайл, юзернейм, эмоджи
             df['tweet'][116], # эмоджи
             df['tweet'][270], # смайл
             
             df['tweet'][232] # сокращенная ссылка
            ]

In [10]:
#for tweet in test_data:
#    print(tweet)

## Настраиваем токенизатор

In [11]:
tokenizer = CrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='splits', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=True, neg_emojis=True, neutral_emojis=True, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops # удаляет стоп слова (разобраться почему падает на кастомном списке)
                          )

## Добавляем еще несколько удалений и замен

In [72]:
def process(tweet, log = False) -> str:
    tweet = re.sub('(\w)\.([А-ЯЁ])', r'\1. \2', tweet) # ставим пробелы
    tweet = re.sub('-', ' ', tweet) # удаляем меняем деффисы на пробел
    if log:
        #print('!до токенизации')
        print(tweet)
    # токенизируем (см. удаления и замены выше)
    tokens = tokenizer.tokenize(tweet)
    if log:
        #print('!после токенизации')
        print(tokens)
        
    new_tokens = []
    for token in tokens:
        if re.search('\d', token): # удаляем все токены, содержащие цифры
            continue
        elif re.search('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token): #удаляем все токены, содержащие оставшиеся символы
            continue
        elif len(token) < 2:
            continue
        new_tokens.append(token)
            
    if log:
        #print('!после обработки каждого токена')
        print(new_tokens)
        #print('!output')
    
    return ' '.join(new_tokens)

In [73]:
for tweet in test_data:
    print(tweet)
    print(process(tweet, log=True), '\n')

Все эти слюнтяи, утверждающие, будто суицид-удел слабых людей, найдут тысячу причин не глотать горсть прозака.Им просто не хватает смелости.
Все эти слюнтяи, утверждающие, будто суицид удел слабых людей, найдут тысячу причин не глотать горсть прозака. Им просто не хватает смелости.
['слюнтяи', 'утверждающие', 'суицид', 'удел', 'слабых', 'людей', 'найдут', 'тысячу', 'причин', 'глотать', 'горсть', 'прозака', 'хватает', 'смелости']
['слюнтяи', 'утверждающие', 'суицид', 'удел', 'слабых', 'людей', 'найдут', 'тысячу', 'причин', 'глотать', 'горсть', 'прозака', 'хватает', 'смелости']
слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин глотать горсть прозака хватает смелости 

Это вам не лыжи,сальбутамол не поможет. Россия - Норвегия 5:1
Это вам не лыжи,сальбутамол не поможет. Россия   Норвегия 5:1
['лыжи', 'сальбутамол', 'поможет', 'россия', 'норвегия', '5:1']
['лыжи', 'сальбутамол', 'поможет', 'россия', 'норвегия']
лыжи сальбутамол поможет россия норвегия 

Если оланзапин мне н

['каждый', 'новыми', 'таблетками', 'зовут', 'паксил', 'надеюсь', 'оправдаешь', 'своё', 'название', 'дашь', 'пак', 'сил', 'паксил', 'держи', 'трясучечку']
['каждый', 'новыми', 'таблетками', 'зовут', 'паксил', 'надеюсь', 'оправдаешь', 'своё', 'название', 'дашь', 'пак', 'сил', 'паксил', 'держи', 'трясучечку']
каждый новыми таблетками зовут паксил надеюсь оправдаешь своё название дашь пак сил паксил держи трясучечку 

Доброе утро коллеги ! Кофеин состоит из углерода, водорода, азота и кислорода — так же, как и кокаин, талидомид, нейлон, тротил и героин.  Приятного кофепития. https://t.co/1XmD9manJj
Доброе утро коллеги ! Кофеин состоит из углерода, водорода, азота и кислорода — так же, как и кокаин, талидомид, нейлон, тротил и героин.  Приятного кофепития. https://t.co/1XmD9manJj
['доброе', 'утро', 'коллеги', 'кофеин', 'состоит', 'углерода', 'водорода', 'азота', 'кислорода', 'кокаин', 'талидомид', 'нейлон', 'тротил', 'героин', 'приятного', 'кофепития']
['доброе', 'утро', 'коллеги', 'кофеин'

## Лемматизация

# Полный скрипт препроцессинга

In [74]:
with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")
    
new_tweets = []
for tweet in tqdm(df['tweet']):
    new_tweets.append(process(tweet, log=False))
    
df['tweet'] = new_tweets

out_file = 'simple_clean_fixed.tsv'

with open('../Data/{}'.format(out_file), 'w', encoding='utf-8') as output:
    df.to_csv(output, sep='\t', index=False, line_terminator='\n')
    
print('Done! Your data is in {}'.format(out_file))

100%|█████████████████████████████████████████████████████████████████████████████| 6090/6090 [00:09<00:00, 647.51it/s]


Done! Your data is in simple_clean_fixed.tsv


In [75]:
df.head(10)

,tweet_id,tweet,class
0,760402871867367424,настало время ингаляторов дружок сальбутамол,0
1,1035908416869462016,прошлой зимней олимпиаде большинство лыжников ...,1
2,1089839736427032577,соглашусь заменой ЗОК метопролол таком виде пе...,0
3,779671488748224513,ЮЗЕРНЕЙМ мезим смекта отравление лоперамид,0
4,738309299756240897,уберите микроволновки имодиум действуют соулмэйты,0
5,1202310351943032834,ЮЗЕРНЕЙМ видео снято время приема оланзапина,0
6,933680693380571136,кроме выстраданные стандарты определения эффек...,0
7,738771297228951552,отчаянно пытаюсь найти тату молекулой флуоксетина,0
8,688135039578550272,слюнтяи утверждающие суицид удел слабых людей ...,0
9,966241801706319878,лыжи сальбутамол поможет россия норвегия,0
